In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

path = [
    "files/apostila.pdf",
    "files/LLM.pdf"
]

pages = []

for p in path:
    loader = PyPDFLoader(p)
    docs = loader.load()
    pages.extend(docs)


recur_char_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]
)

split_docs = recur_char_split.split_documents(pages)

directory = "files/chroma_vectorstore"

embeddings = OpenAIEmbeddings()
vector_store = Chroma.from_documents(
    split_docs, embeddings, persist_directory=directory
)



In [ ]:
for i, doc in enumerate(split_docs):
    doc.metadata['source'] = doc.metadata['source'].replace('files/', '')
    doc.metadata['doc_id'] = i

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

embeddings_model = OpenAIEmbeddings()
diretorio = 'files/chroma_retrival_bd'

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings_model,
    persist_directory=diretorio
)

In [ ]:
# Semantic search
question = "O que é uma LLM?"
docs = vectordb.similarity_search(question, k=10)
for doc in docs:
    print(f"Source: {doc.metadata}")
    print(f"Content: {doc.page_content}\n")

In [ ]:
# Max Marginal Relevance -> Ele é usado para trazer os documentos mais relevantes e diversos
question = "O que é uma LLM?"
# k é o número de documentos que você quer retornar
# fetch_k é o número de documentos que serão buscados inicialmente para depois aplicar a MMR
docs = vectordb.max_marginal_relevance_search(question, k=3, fetch_k=10) 
for doc in docs:
    print(f"Source: {doc.metadata}")
    print(f"Content: {doc.page_content}\n")

In [ ]:
# Filtragem
question = "O que a apostila de LLM fala sobre a OpenAI e o ChatGPT?"
docs = vectordb.similarity_search(question, k=3, filter={"source": "apostila.pdf"}) 
# docs = vectordb.similarity_search(
#     question, 
#     k=3,
#     filter={'$and':
#             [{'source': {'$in': ['LLM.pdf']}},
#             {'page': {'$in': [3, 4, 5, 6]}}],
#             })
for doc in docs:
    print(f"Source: {doc.metadata}")
    print(f"Content: {doc.page_content}\n")

In [ ]:
### LLM Aided Retrieval
from langchain_openai.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo

metadata_info = [
    AttributeInfo(
        name='source',
        description='Nome da apostila de onde o texto original foi retirado. Pode ser "apostila.pdf" ou "LLM.pdf".',
        type='string'
    ),
    AttributeInfo(
        name='page',
        description='A página da apostila de onde o texto foi extraído. Número da página.',
        type='integer'
    ),
]

document_description = 'Apostilas de informações'
llm = OpenAI()
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_description,
    metadata_info,
    verbose=True
)

pergunta = 'O que a apostila de LLM fala sobre a OpenAI, ChatGPT e Hugging Face?'

docs = retriever.get_relevant_documents(pergunta)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')
